# 查看显卡类型（是否成功安装）以及挂载谷歌云盘

In [ ]:
! nvidia-smi

from google.colab import drive
drive.mount('/content/gdrive')

# 引用模块

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import cv2

# 定义CNN的架构

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 第一个卷积层，输入通道数为1，输出通道数为32，卷积核大小为3x3，padding大小为1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        # 第二个卷积层，输入通道数为32，输出通道数为64，卷积核大小为3x3，padding大小为1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # 第一个全连接层，输入大小为64*7*7，输出大小为128
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        # 第二个全连接层，输入大小为128，输出大小为10
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        # 输入x的大小为[batch_size, 1, 28, 28]
        # 第一次卷积，使用ReLU作为激活函数
        x = nn.functional.relu(self.conv1(x)) # 输出大小为[batch_size, 32, 28, 28]
        # 第一次池化，使用2x2的最大池化操作
        x = nn.functional.max_pool2d(x, 2) # 输出大小为[batch_size, 32, 14, 14]
        # 第二次卷积，使用ReLU作为激活函数
        x = nn.functional.relu(self.conv2(x)) # 输出大小为[batch_size, 64, 14, 14]
        # 第二次池化，使用2x2的最大池化操作
        x = nn.functional.max_pool2d(x, 2) # 输出大小为[batch_size, 64, 7, 7]
        # 将特征张量展开成一维张量
        x = x.view(-1, 64 * 7 * 7) # 输出大小为[batch_size, 64*7*7]
        # 第一个全连接层，使用ReLU作为激活函数
        x = nn.functional.relu(self.fc1(x)) # 输出大小为[batch_size, 128]
        # 第二个全连接层，不使用激活函数
        x = self.fc2(x) # 输出大小为[batch_size, 10]
        return x


# 定义超参数与数据加载器（从数据集中加载批量的图像）

In [ ]:
# 定义超参数：

# 每个批次的图像数量
batch_size = 64
# 学习率
learning_rate = 0.01
# 训练轮数
num_epochs = 10


# 定义数据加载器（从数据集中加载批量的图像）

# 使用transforms.Compose定义了一个数据预处理管道，将图像转换为张量
# 对像素值进行归一化，将值从[0,1]范围缩放到[-1,1]范围。
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])


# 使用PyTorch中的datasets.MNIST加载MNIST数据集，并使用DataLoader将数据分成批次进行训练。
# 每个批次的图像数量为batch_size，shuffle=True表示每个批次都是随机的。

# train_loader用于训练
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# test_loader用于测试。
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



# 训练模型

使用nn.CrossEntropyLoss()损失函数。
使用optim.SGD优化器。

接下来，使用for循环进行训练。外层循环迭代num_epochs次，内层循环迭代train_loader中的每个批次。在每个批次中，images和labels分别为图像张量和对应标签的张量。将images和labels转换为float和long数据类型，然后将优化器的梯度置零。使用网络模型net对图像进行预测，得到输出outputs。使用损失函数criterion计算预测输出outputs与真实标签labels之间的损失loss。调用loss.backward()方法自动计算梯度并反向传播，然后调用optimizer.step()方法更新网络参数。

在训练过程中，每100个批次打印一次训练信息，包括当前轮数、训练轮数、当前批次、总批次数以及当前批次的损失值。

In [ ]:
# 使用交叉熵损失函数和随机梯度下降优化器
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        labels = labels.long()

        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, len(train_loader), loss.item()))


# 测试模型

这段代码的功能是评估训练好的神经网络在测试数据集上的准确率。

初始化了两个变量 correct 和 total，用于记录测试集中正确预测的数量和测试集总共的图片数量。

然后，我们使用 torch.no_grad() 上下文来关闭梯度计算，在评估过程中我们不需要计算梯度。

在上下文中，迭代测试数据集，将每个批次的图像通过网络，并将预测的标签与真实标签进行比较，以计算正确预测的数量。

最后，我们打印出网络在测试集上的准确率。

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.float()
        labels = labels.long()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {:.2f} %'.format(100 * correct / total))